# Transformers 模型量化技术：AWQ

![img](https://huggingface.co/datasets/ybelkada/documentation-images/resolve/main/Thumbnail.png)

在2023年6月，Ji Lin等人发表了论文[AWQ：Activation-aware Weight Quantization for LLM Compression and Acceleration](https://arxiv.org/pdf/2306.00978.pdf)。

这篇论文详细介绍了一种激活感知权重量化算法，可以用于压缩任何基于 Transformer 的语言模型，同时只有微小的性能下降。关于 AWQ 算法的详细介绍，见[MIT Han Song 教授分享](https://hanlab.mit.edu/projects/awq)。

transformers 现在支持两个不同的 AWQ 开源实现库：

- [AutoAWQ](https://github.com/casper-hansen/AutoAWQ)
- [LLM-AWQ](https://github.com/mit-han-lab/llm-awq) 


因为 LLM-AWQ 不支持 Nvidia T4 GPU（课程演示 GPU），所以我们使用 AutoAWQ 库来介绍和演示 AWQ 模型量化技术。

## 量化前模型测试文本生成任务

In [1]:
from transformers import pipeline

model_path = "facebook/opt-125m"

# 使用 GPU 加载原始的 OPT-125m 模型
generator = pipeline('text-generation',
                     model=model_path,
                     device=0,
                     do_sample=True,
                     num_return_sequences=3)

/root/miniconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


#### 实测GPU显存占用：加载 OPT-125m 模型后

```shell
Sun Dec 24 15:11:33 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   47C    P0              26W /  70W |    635MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
```

In [2]:
generator("The woman worked as a")

[{'generated_text': 'The woman worked as a school nurse and the man was a teacher. The men were the teachers!'},
 {'generated_text': 'The woman worked as a waitress, waitress, barista and food truck driver, and was an early'},
 {'generated_text': 'The woman worked as a security guard at a motel in her hometown in San Francisco. Photograph: T'}]

In [3]:
generator("The man worked as a")

[{'generated_text': 'The man worked as a delivery boy before. He made a fortune as a courier and it makes no'},
 {'generated_text': 'The man worked as a taxi driver for five years. He had a lot to do, and there'},
 {'generated_text': 'The man worked as a security guard at a gas station at the time and was a co-worker'}]

## 使用 AutoAWQ 量化模型

下面我们以 `facebook opt-125m` 模型为例，使用 `AutoAWQ` 库实现的 AWQ 算法实现模型量化。

In [4]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


quant_path = "models/opt-125m-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

/root/miniconda3/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 12/12 [01:19<00:00,  6.59s/it]


#### 实测GPU显存使用：量化模型时峰值达到将近 4GB

```shell
Sun Dec 24 15:12:50 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   48C    P0              32W /  70W |    3703MiB / 15360MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
```

In [6]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

#### Transformers 兼容性配置

为了使`quant_config` 与 transformers 兼容，我们需要修改配置文件：`使用 Transformers.AwqConfig 来实例化量化模型配置`

In [7]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [8]:
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)  # 保存分词器

('models/opt-125m-awq/tokenizer_config.json',
 'models/opt-125m-awq/special_tokens_map.json',
 'models/opt-125m-awq/vocab.json',
 'models/opt-125m-awq/merges.txt',
 'models/opt-125m-awq/added_tokens.json',
 'models/opt-125m-awq/tokenizer.json')

### 使用 GPU 加载量化模型

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="cuda").to(0)

In [10]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)


In [11]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

Merry Christmas! I'm glad to hear you're doing well!
Thank you! I'm glad to hear you're doing well!


In [12]:
result = generate_text("The woman worked as a")
print(result)

The woman worked as a nurse at the hospital for a year. She was a nurse at the hospital for a year. She was a nurse at the hospital for a year. She was a nurse at the hospital for a year. She was a nurse at the hospital for a year. She was a nurse at the hospital for a year. She


## Homework：使用 AWQ 量化 Facebook OPT-6.7B 模型

Facebook OPT 模型：https://huggingface.co/facebook?search_models=opt

In [1]:
from transformers import pipeline

model_path_opt67b = "facebook/opt-6.7b"

# 使用 GPU 加载原始的 OPT-125m 模型
#generator = pipeline('text-generation',
#                     model=model_path,
#                     device=0,
#                     do_sample=True,
#                     num_return_sequences=3)

/root/miniconda3/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


quant_path_opt67b = "models/opt-6.7b-awq"
quant_config_opt67b = {"zero_point": True, "q_group_size": 128, "w_bit": 2, "version": "GEMM"}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_path_opt67b, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path_opt67b, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.27s/it]


In [3]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config_opt67b)

/root/miniconda3/envs/llmenv/lib/python3.10/site-packages/huggingface_hub-0.20.1-py3.8.egg/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
AWQ:   0%|          | 0/32 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 960.00 MiB. GPU 0 has a total capacty of 14.58 GiB of which 289.75 MiB is free. Including non-PyTorch memory, this process has 14.29 GiB memory in use. Of the allocated memory 14.15 GiB is allocated by PyTorch, and 12.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
quant_config_opt67b

In [ ]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config_opt67b = AwqConfig(
    bits=quant_config_opt67b["w_bit"],
    group_size=quant_config_opt67b["q_group_size"],
    zero_point=quant_config_opt67b["zero_point"],
    version=quant_config_opt67b["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config_opt67b

In [ ]:
# 保存模型权重
model.save_quantized(quant_path_opt67b)
tokenizer.save_pretrained(quant_path_opt67b)  # 保存分词器